In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
import random

In [2]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, RNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import pandas as pd
np.random.seed(1)

Using TensorFlow backend.


### Helper functions for Data Loading, normalization, etc

In [3]:
def Reshape_to_input(X, T, Mat_dim):
    """
    reshapes data
    """
   
    [Num_examples, dim] = X.shape
    input_X = X[:,2:].reshape(Num_examples, Mat_dim, T)
    
    return input_X
    

In [4]:
def NormalizeTowardsMean(X):
   # print(X.shape)
    "# Preprocessing: Subtract the mean feature - a question if it is feasible - all features are already in the same scale...\n",
    # TODO: try -1 1 normalization
    mean_feat = np.mean(X, axis=0, keepdims=True)
   # print(mean_feat.shape)
    X -= mean_feat
    # Preprocessing: Divide by standard deviation. This ensures that each feature\n",
    # has roughly the same scale.\n",
    std_feat = np.std(X, axis=0, keepdims=True)
    X /= std_feat
    
    return X



In [5]:
def Random_Selection_train_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    Persons_train = np.array([])
    Persons_test = np.array([])
    #print(X_train.shape)
    for Person in range(0,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, returned_train
            
            


In [6]:
def Random_Selection_train_val_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    X_val = np.array([])
    Y_val = np.array([])
    Persons_train = np.array([])
    Persons_val = np.array([])
    Persons_test = np.array([])
    
    train_N =int(np.ceil(N*2/3))
    
    for Person in range(0,train_N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    
     
    for Person in range(train_N,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_val= np.append(X_val,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_val = np.append(Y_val, X_all[[indexes], 0])
        Persons_val = np.append(Persons_val, label, axis = 0)
    X_val=X_val.reshape(Y_val.size, num_dim)  
    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for val: ')
    print(np.unique(Persons_val))
    returned_val = np.unique(Persons_val)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val
            
            


In [7]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, int(inputY[i]-1)] = 1
        
    
    return Y


In [8]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    units = 128 
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = LSTM(units, return_sequences=True)( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = LSTM(units, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [9]:
# load my dataset and parameters from a file
from data_utils import loadfromfolder

### MODEL 2 SHORT

In [36]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    units =  192
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = LSTM(units, return_sequences=False)( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    #X = LSTM(units, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    #X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

### Model 3 LONG


In [29]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    units =  192
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = LSTM(units, return_sequences=True)( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    X = LSTM(units, return_sequences=True)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = LSTM(units, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

### MAIN Loop through all the files 

In [ ]:
# parameters needed
trials = 7
epochs = 35
batch_size = 15
units = 128 # actually defind inside the funciton of the model - TODO - make it here
path ='data/3_angles'
Files, T, max_dim = loadfromfolder()
#Where Files are names, T - my division into frames and max_dim - feature dimensionality
fRes = open("Results.txt","a+") 
fRes.write("Number of units in LSTM = %d,  Epochs =  %d and batch size =  %d \r\n" % (units, epochs,batch_size))
for file in range(0, len(Files)):
    name = Files[file]
    t = T[file]
    dim = max_dim[file]
    name_w_path = path +"/"+name
    fRes.write("Experiment %d from file %s \r\n" % (file,name))
    # here all the routine in One cell 
    X_all = pd.read_csv(name_w_path)
    
    # the routine to run the same test N times, randomly shuffling the data
    Accuracy = X_train = np.zeros(shape=(trials))
    for i in range(0,trials):
        X_train, Y_train, X_test, Y_test, returned_train = Random_Selection_train_test (X_all.values, 14)
        #fRes.write(returned_train) # the presons used for training
        namePersons = str(i)+'_'+'file'+ name[:-4] +'.txt'
        np.savetxt(namePersons, returned_train, delimiter=',', fmt='%d')   # X is an array
        X_train = NormalizeTowardsMean(X_train)
        X_test = NormalizeTowardsMean(X_test)
        Y_train_oh = convert_to_one_hot(Y_train, C = 3)
        Y_test_oh = convert_to_one_hot(Y_test, C = 3)
        X_train = Reshape_to_input(X_train, t, dim)
        X_test = Reshape_to_input(X_test, t, dim)
        model = Gait_model((X_train[0,:,:]))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, Y_train_oh, epochs, batch_size, shuffle=True)
        loss, acc = model.evaluate(X_test, Y_test_oh)
        Accuracy[i]=acc
        fRes.write("Final accuracy per trial %f \r\n" % (acc))
        
    print(Accuracy)
    fRes.write("Final mean accuracy is: %f \r\n" % (np.mean(Accuracy)))
fRes.close() 


### Model 2

In [10]:
myoptim=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filepath, save_best_only =True, monitor = 'val_loss', mode ='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min')
    return [es, mcp_save, reduce_lr_loss]

In [ ]:
myoptim=SGD(lr=0.01, decay=1e-8, momentum=0.9, nesterov=True)

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filepath, save_best_only =True, monitor = 'val_loss', mode ='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min')
    return [es, mcp_save, reduce_lr_loss]

In [11]:
# parameters needed
trials = 10
epochs = 40
batch_size = 30
units = 192 # actually defind inside the funciton of the model - TODO - make it here
path ='data/best'
Files, T, max_dim = loadfromfolder()
#Where Files are names, T - my division into frames and max_dim - feature dimensionality
fRes = open("Results.txt","a+") 
fRes.write("Number of units in LSTM = %d,  Epochs =  %d and batch size =  %d \r\n" % (units, epochs,batch_size))
for file in range(0, len(Files)):
    name = Files[file]
    t = T[file]
    dim = max_dim[file]
    name_w_path = path +"/"+name
    fRes.write("Experiment %d from file %s \r\n" % (file,name))
    # here all the routine in One cell 
    print(name_w_path)
    X_all = pd.read_csv(name_w_path)
    # the routine to run the same test N times, randomly shuffling the data
    Accuracy  = np.zeros(shape=(trials))
   
    
    for i in range(0,trials):
        modelname = str(file)+str(i)
        X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val = Random_Selection_train_val_test (X_all.values, 16)
        #namePersons = str(i)+'_'+'file'+ name[:-4] +'.txt'
        #np.savetxt(namePersons, returned_train, delimiter=',', fmt='%d')   # X is an array
        X_train = NormalizeTowardsMean(X_train)
        X_test = NormalizeTowardsMean(X_test)
        X_val = NormalizeTowardsMean(X_val)
        
        Y_train_oh = convert_to_one_hot(Y_train, C = 3)
        Y_test_oh = convert_to_one_hot(Y_test, C = 3)
        Y_val_oh = convert_to_one_hot(Y_val, C = 3)
        
        X_train = Reshape_to_input(X_train, t, dim)
        X_test = Reshape_to_input(X_test, t, dim)
        X_val = Reshape_to_input(X_val, t, dim)
        
        model = Gait_model((X_train[0,:,:]))
        
        file_path = 'Models_LSTM/file_{}_model_wts.hdf5'.format(modelname)
        callbacks = get_callbacks(filepath=file_path, patience = 35)
        model.compile(loss='categorical_crossentropy', optimizer=myoptim, metrics=['accuracy'])
              
        
        model.fit(X_train, Y_train_oh, epochs=epochs,\
                  batch_size=batch_size, shuffle=True, validation_data=(X_val, Y_val_oh), callbacks=callbacks)
        
        loss, acc = model.evaluate(X_test, Y_test_oh)
        
        Accuracy[i]=acc
        fRes.write("Final accuracy per trial %f \r\n" % (acc))
        
    print(Accuracy)
    fRes.write("Final mean accuracy is: %f \r\n" % (np.mean(Accuracy)))
fRes.close() 


angles_cycles_frames_15t_08f_angles_22persons.csv
data/best/angles_cycles_frames_15t_08f_angles_22persons.csv
Persons for train: 
[ 1.  2.  6.  7.  9. 10. 11. 15. 16. 19. 22.]
Persons for val: 
[ 4.  8. 18. 20. 21.]
Persons for test: 
[ 3.  5. 12. 13. 14. 17.]


/home/khokhlov/virtenv/ml36/lib/python3.5/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 280 samples, validate on 124 samples
Epoch 1/40
280/280 [==============================] - 4s 14ms/step - loss: 1.0917 - acc: 0.3500 - val_loss: 1.0866 - val_acc: 0.4919
Epoch 2/40
280/280 [==============================] - 0s 1ms/step - loss: 1.0674 - acc: 0.5179 - val_loss: 1.0630 - val_acc: 0.5645
Epoch 3/40
280/280 [==============================] - 0s 2ms/step - loss: 1.0385 - acc: 0.5643 - val_loss: 1.0398 - val_acc: 0.5645
Epoch 4/40
280/280 [==============================] - 0s 2ms/step - loss: 1.0091 - acc: 0.5571 - val_loss: 1.0140 - val_acc: 0.5806
Epoch 5/40
280/280 [==============================] - 0s 2ms/step - loss: 0.9667 - acc: 0.6357 - val_loss: 0.9851 - val_acc: 0.5565
Epoch 6/40
280/280 [==============================] - 0s 2ms/step - loss: 0.9299 - acc: 0.6286 - val_loss: 0.9508 - val_acc: 0.5887
Epoch 7/40
280/280 [==============================] - 0s 2ms/step - loss: 0.8771 - acc: 0.7036 - val_loss: 0.9148 - val_acc: 0.5968
Epoch 8/40
280/280 [=========

Epoch 34/40
284/284 [==============================] - 0s 1ms/step - loss: 0.8155 - acc: 0.6796 - val_loss: 0.8879 - val_acc: 0.6364
Epoch 35/40
284/284 [==============================] - 0s 1ms/step - loss: 0.8083 - acc: 0.6831 - val_loss: 0.8842 - val_acc: 0.6364
Epoch 36/40
284/284 [==============================] - 0s 1ms/step - loss: 0.8004 - acc: 0.7007 - val_loss: 0.8812 - val_acc: 0.6364
Epoch 37/40
284/284 [==============================] - 0s 1ms/step - loss: 0.7967 - acc: 0.7183 - val_loss: 0.8782 - val_acc: 0.6446
Epoch 38/40
284/284 [==============================] - 0s 2ms/step - loss: 0.7872 - acc: 0.6901 - val_loss: 0.8756 - val_acc: 0.6446
Epoch 39/40
284/284 [==============================] - 0s 1ms/step - loss: 0.7879 - acc: 0.7042 - val_loss: 0.8730 - val_acc: 0.6446
Epoch 40/40
142/142 [==============================] - 0s 410us/step
Persons for train: 
[ 1.  2.  7.  9. 11. 14. 15. 16. 19. 21. 22.]
Persons for val: 
[ 4.  5.  8. 10. 12.]
Persons for test: 
[ 3.  6.

Epoch 12/40
265/265 [==============================] - 0s 1ms/step - loss: 0.9463 - acc: 0.6755 - val_loss: 0.9807 - val_acc: 0.5909
Epoch 13/40
265/265 [==============================] - 0s 1ms/step - loss: 0.9344 - acc: 0.7094 - val_loss: 0.9708 - val_acc: 0.6000
Epoch 14/40
265/265 [==============================] - 0s 1ms/step - loss: 0.9244 - acc: 0.6679 - val_loss: 0.9605 - val_acc: 0.6000
Epoch 15/40
265/265 [==============================] - 0s 1ms/step - loss: 0.9122 - acc: 0.7019 - val_loss: 0.9507 - val_acc: 0.6091
Epoch 16/40
265/265 [==============================] - 0s 2ms/step - loss: 0.9028 - acc: 0.6755 - val_loss: 0.9397 - val_acc: 0.6182
Epoch 17/40
265/265 [==============================] - 0s 2ms/step - loss: 0.8930 - acc: 0.6906 - val_loss: 0.9290 - val_acc: 0.6273
Epoch 18/40
265/265 [==============================] - 0s 2ms/step - loss: 0.8718 - acc: 0.6868 - val_loss: 0.9188 - val_acc: 0.6273
Epoch 19/40
265/265 [==============================] - 0s 2ms/step - 

Epoch 32/40
286/286 [==============================] - 0s 2ms/step - loss: 0.7815 - acc: 0.6469 - val_loss: 0.9047 - val_acc: 0.5810
Epoch 33/40
286/286 [==============================] - 0s 1ms/step - loss: 0.7701 - acc: 0.6538 - val_loss: 0.9048 - val_acc: 0.5810
Epoch 34/40
286/286 [==============================] - 0s 2ms/step - loss: 0.7723 - acc: 0.6713 - val_loss: 0.9049 - val_acc: 0.5810
Epoch 35/40
286/286 [==============================] - 0s 2ms/step - loss: 0.7726 - acc: 0.6818 - val_loss: 0.9052 - val_acc: 0.5714
Epoch 36/40
286/286 [==============================] - 0s 2ms/step - loss: 0.7661 - acc: 0.6503 - val_loss: 0.9054 - val_acc: 0.5714
Epoch 37/40
286/286 [==============================] - 0s 1ms/step - loss: 0.7600 - acc: 0.6573 - val_loss: 0.9062 - val_acc: 0.5714
Epoch 38/40
286/286 [==============================] - 0s 1ms/step - loss: 0.7595 - acc: 0.6608 - val_loss: 0.9068 - val_acc: 0.5714
Epoch 39/40
286/286 [==============================] - 0s 1ms/step - 

Epoch 10/40
258/258 [==============================] - 0s 2ms/step - loss: 0.9431 - acc: 0.6744 - val_loss: 0.9840 - val_acc: 0.5000
Epoch 11/40
258/258 [==============================] - 0s 2ms/step - loss: 0.9207 - acc: 0.6822 - val_loss: 0.9749 - val_acc: 0.5000
Epoch 12/40
258/258 [==============================] - 0s 2ms/step - loss: 0.9118 - acc: 0.6783 - val_loss: 0.9664 - val_acc: 0.5000
Epoch 13/40
258/258 [==============================] - 0s 2ms/step - loss: 0.8987 - acc: 0.6938 - val_loss: 0.9584 - val_acc: 0.5000
Epoch 14/40
258/258 [==============================] - 0s 2ms/step - loss: 0.8868 - acc: 0.6550 - val_loss: 0.9513 - val_acc: 0.5149
Epoch 15/40
258/258 [==============================] - 0s 2ms/step - loss: 0.8703 - acc: 0.6938 - val_loss: 0.9452 - val_acc: 0.5000
Epoch 16/40
258/258 [==============================] - 0s 2ms/step - loss: 0.8597 - acc: 0.7093 - val_loss: 0.9400 - val_acc: 0.5149
Epoch 17/40
258/258 [==============================] - 0s 2ms/step - 

257/257 [==============================] - 0s 2ms/step - loss: 1.0726 - acc: 0.4397 - val_loss: 1.0704 - val_acc: 0.5547
Epoch 30/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0673 - acc: 0.5019 - val_loss: 1.0699 - val_acc: 0.5547
Epoch 31/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0656 - acc: 0.5019 - val_loss: 1.0693 - val_acc: 0.5547
Epoch 32/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0661 - acc: 0.4981 - val_loss: 1.0688 - val_acc: 0.5620
Epoch 33/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0707 - acc: 0.4786 - val_loss: 1.0683 - val_acc: 0.5693
Epoch 34/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0716 - acc: 0.4280 - val_loss: 1.0679 - val_acc: 0.5839
Epoch 35/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0639 - acc: 0.5058 - val_loss: 1.0674 - val_acc: 0.5839
Epoch 36/40
257/257 [==============================] - 0s 2ms/step - loss: 1.0625

Epoch 7/40
276/276 [==============================] - 0s 2ms/step - loss: 1.0774 - acc: 0.4457 - val_loss: 1.0864 - val_acc: 0.4865
Epoch 8/40
276/276 [==============================] - 0s 2ms/step - loss: 1.0719 - acc: 0.5181 - val_loss: 1.0855 - val_acc: 0.5045
Epoch 9/40
276/276 [==============================] - 0s 2ms/step - loss: 1.0734 - acc: 0.4964 - val_loss: 1.0847 - val_acc: 0.5045
Epoch 10/40
276/276 [==============================] - 0s 2ms/step - loss: 1.0700 - acc: 0.5036 - val_loss: 1.0838 - val_acc: 0.5045
Epoch 11/40
276/276 [==============================] - 0s 1ms/step - loss: 1.0698 - acc: 0.5254 - val_loss: 1.0830 - val_acc: 0.5045
Epoch 12/40
276/276 [==============================] - 0s 1ms/step - loss: 1.0658 - acc: 0.5217 - val_loss: 1.0823 - val_acc: 0.5045
Epoch 13/40
276/276 [==============================] - 0s 1ms/step - loss: 1.0681 - acc: 0.5145 - val_loss: 1.0815 - val_acc: 0.5045
Epoch 14/40
276/276 [==============================] - 0s 2ms/step - los

In [ ]:
fRes.close() 